In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
import json
from search.python_utils import stringify, unstringify
from copy import deepcopy
from coderm.execution import smart_exec_tests_queuebatched

In [ ]:
data = load_dataset("evalplus/mbppplus")

In [ ]:
test_data = data["test"]

In [ ]:
results = smart_exec_tests_queuebatched(test_data["code"], test_data["test"])
[i for i, (stat, _) in enumerate(results) if not stat]

In [ ]:
test_dicts = []
public_test_dicts = []

for i, row in enumerate(test_data):
    fn_name = row["test"].splitlines()[-1].split("assertion(")[1].split("(*inp")[0].strip()
    test_dicts.append({"inputs": [], "outputs": [], "fn_name": fn_name, "exec_string": row["test"].strip()})
    public_test_dicts.append({"inputs": [], "outputs": [], "fn_name": fn_name, "exec_string": '\n'.join(row["test_list"])})

In [ ]:
codes = [c.strip() for c in test_data["code"]]

In [ ]:
results = smart_exec_tests_queuebatched(codes, [t["exec_string"] for t in test_dicts])
[i for i, (stat, _) in enumerate(results) if not stat]

In [ ]:
results = smart_exec_tests_queuebatched(codes, [t["exec_string"] for t in public_test_dicts])
[i for i, (stat, _) in enumerate(results) if not stat]

In [ ]:
def retrieve_starter_code(s: str, fn_name: str) -> str:
    starter_code = None
    for line in s.splitlines():
        if fn_name in line:
            starter_code = line.split(":")[0].strip() + ":\n    "
            break
    assert starter_code is not None and starter_code.startswith("def ")
    return starter_code

In [ ]:
print(codes[0])
retrieve_starter_code(codes[0], test_dicts[0]["fn_name"])

In [ ]:
print(test_data["prompt"][0] + f"\n\nYour code should pass these tests:\n```\n{public_test_dicts[0]['exec_string']}\n```")

In [ ]:
newline = '\n'
new_test_data = deepcopy(test_data)
new_test_data= new_test_data.rename_column("task_id", "id")
new_test_data = new_test_data.map(lambda row, idx: {"starter_code": retrieve_starter_code(codes[idx], test_dicts[idx]["fn_name"])}, with_indices=True)
new_test_data = new_test_data.map(lambda row, idx: {"solutions": [codes[idx]]}, with_indices=True)
new_test_data = new_test_data.map(lambda row, idx: {"public_input_output": repr(public_test_dicts[idx])}, with_indices=True)
new_test_data = new_test_data.map(lambda row, idx: {"input_output": repr(test_dicts[idx])}, with_indices=True)
new_test_data = new_test_data.map(lambda row, idx: {"question": row['prompt'].strip() + f"\n\nYour code should pass these tests:\n```\n{public_test_dicts[idx]['exec_string']}\n```"}, with_indices=True)
new_test_data = new_test_data.remove_columns(["code", "prompt", "source_file", "test_imports", "test_list", "test",])
new_test_data

In [ ]:
dd = DatasetDict({"test": new_test_data})
dd.push_to_hub("codegenning/F_mbpp_plus", private=True)